In [42]:
import bs4
import requests
import json
from random import shuffle
from thefuzz import process, fuzz
from modules.CountMinSketch.count_min_sketch import CountMinSketch
from modules.Dataset.dataset import Dataset

In [43]:
ds = Dataset()

words_list = ds.build_words_list(num_of_articles = 12)
print(len(words_list))

264063


In [44]:
len(words_list)

264063

In [45]:
NUM_COUNTERS = 200000
cms = CountMinSketch(NUM_COUNTERS)

for word in words_list:
    cms.increment(word)

In [46]:
#some_word_with_error = "appel"
some_sentance_with_error = "I wanna apple pleases or a bannana with a paer and a lemone, or a wartermelon if possibe"
new_sentance = []
for word in some_sentance_with_error.split():
    success_percentage = max((len(word) - 1) / len(word) * 100, 75)
    print(f"Scanning {word}, success_percentage is {success_percentage}")
    word_results = [x for x in process.extract(word, words_list, scorer=fuzz.ratio) if x[1] >= success_percentage]
    if (len(word_results) > 0 and word_results[0][1] < 100):
        list_fixes = [x[0] for x in word_results]
        for fix in list_fixes:
            print(f"Frequency of {fix} => {cms.approximateCount(fix)}")
        #new_sentance.append(word_results[0][0] if word_results[0][1] >= success_percentage else word)
    else:
        continue
        #new_sentance.append(word)

#print(' '.join(new_sentance))

Scanning I, success_percentage is 75
Scanning wanna, success_percentage is 80.0
Frequency of Anna => 3
Frequency of Anna => 3
Frequency of Anna => 3
Frequency of manna => 21
Frequency of Swann => 3
Scanning apple, success_percentage is 80.0
Scanning pleases, success_percentage is 85.71428571428571
Frequency of Please => 6
Frequency of Please => 6
Frequency of Please => 6
Frequency of Please => 6
Frequency of Pleasures => 1
Scanning or, success_percentage is 75
Scanning a, success_percentage is 75
Scanning bannana, success_percentage is 85.71428571428571
Frequency of bananas. => 1
Scanning with, success_percentage is 75.0
Scanning a, success_percentage is 75
Scanning paer, success_percentage is 75.0
Frequency of Paper" => 1
Frequency of Paper => 2
Frequency of paper => 4
Frequency of paper. => 3
Frequency of Paper => 2
Scanning and, success_percentage is 75
Scanning a, success_percentage is 75
Scanning lemone,, success_percentage is 85.71428571428571
Frequency of Leone => 3
Frequency of